### Постановка задачи
Загрузите данные и посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: в первой модели весь оптимальный набор метеорологических данных, во второй - дни недели и праздники, в третьей - недели года, в четвертой - месяцы. Финальное значение показателя рассчитайте как взвешенное арифметическое показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузите данные решения, посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

Сколько строк в итоговом файле имеют ненулевое значение для показателя meter_reading ?

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/test.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

# Подключение библиотек

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression

### Загрузка данных для расчета, оптимизация памяти
##### В целях оптимизации исходных данных максимально уменьшим объем памяти,  используя функцию оптимизации памяти. Проверим исходный тип данных и уменьшим его размер.

In [2]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

## Загрузка данных для модели по 50 зданиям

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
weather = weather[weather["site_id"] == 0] # выводим данные по столбцу "site_id" со значением 0

In [4]:
print(buildings.columns.tolist ()) # выводим название столбцов "buildings"

['site_id', 'building_id', 'primary_use', 'square_feet', 'year_built', 'floor_count']


In [5]:
print(weather.columns.tolist ()) # выводим название столбцов "weather"

['site_id', 'timestamp', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']


In [7]:
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
print(energy.columns.tolist ()) # выводим название столбцов "energy"

['building_id', 'meter', 'timestamp', 'meter_reading']


In [8]:
energy = energy[energy["building_id"]<50] # отобразим все данные для 50 зданий
print (energy)

          building_id  meter            timestamp  meter_reading
0                   0      0  2016-01-01 00:00:00         0.0000
1                   1      0  2016-01-01 00:00:00         0.0000
2                   2      0  2016-01-01 00:00:00         0.0000
3                   3      0  2016-01-01 00:00:00         0.0000
4                   4      0  2016-01-01 00:00:00         0.0000
...               ...    ...                  ...            ...
12059551           45      0  2016-12-31 23:00:00         2.8668
12059552           46      0  2016-12-31 23:00:00         2.3958
12059553           47      0  2016-12-31 23:00:00       209.6280
12059554           48      0  2016-12-31 23:00:00        25.7223
12059555           49      0  2016-12-31 23:00:00       131.0520

[432456 rows x 4 columns]


In [9]:
# объединим два набора данных "energy" и "buildings". С помощью параметра how="left" мы указали, что главным является 
# датафрейм слева, название столбца – «building_id» 

energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
print(energy.columns.tolist ()) # выводим название столбцов 

['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id', 'primary_use', 'square_feet', 'year_built', 'floor_count']


In [10]:
print (energy.head()) # просмотрим первые пять строк таблицы

   building_id  meter            timestamp  meter_reading  site_id  \
0            0      0  2016-01-01 00:00:00            0.0        0   
1            1      0  2016-01-01 00:00:00            0.0        0   
2            2      0  2016-01-01 00:00:00            0.0        0   
3            3      0  2016-01-01 00:00:00            0.0        0   
4            4      0  2016-01-01 00:00:00            0.0        0   

  primary_use  square_feet  year_built  floor_count  
0   Education         7432      2008.0          NaN  
1   Education         2720      2004.0          NaN  
2   Education         5376      1991.0          NaN  
3   Education        23685      2002.0          NaN  
4   Education       116607      1975.0          NaN  


In [11]:
print (energy.info()) #Посмотрим сводную информацию методом info()
# в таблице 432456 строки (0 to 432455)
# 9 столбцов - количество ненулевых значений (non-null)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432456 entries, 0 to 432455
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   building_id    432456 non-null  int64  
 1   meter          432456 non-null  int64  
 2   timestamp      432456 non-null  object 
 3   meter_reading  432456 non-null  float64
 4   site_id        432456 non-null  int64  
 5   primary_use    432456 non-null  object 
 6   square_feet    432456 non-null  int64  
 7   year_built     432456 non-null  float64
 8   floor_count    0 non-null       float64
dtypes: float64(3), int64(4), object(2)
memory usage: 33.0+ MB
None


###  Интерполяция и обогащение данных по погоде

In [12]:
# Вызовим функцию "weather_interpolate_diff" по параметру "weather"
# Произведем интерполяцию (сглаживание) данных по "weather"
# установим параметры: limit_direction='both' - интерполируются оба конца
# kind='cubic' - точки гладко соединяются кубическими полиномами
# используя функцию diff обогатим данные по погоде
def weather_interpolate_diff (weather):
    interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "sea_level_pressure"]
    for col in interpolate_columns:
        weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')
    weather["air_temperature_diff1"] = weather["air_temperature"].diff()
    weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
    weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
    weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]
    return weather # возвращаем значение "weather"    

### Объединение данных с погодными

In [13]:
energy = energy.set_index(["timestamp", "site_id"]) #проиндексируем фрейм данных по "timestamp", "site_id"
weather = weather.set_index(["timestamp", "site_id"]) #проиндексируем фрейм данных по "timestamp", "site_id"
# объединим данные по "energy" и "weather"
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)# сбросим индекс фрейма данных "energy"
# удалим столбцы с данными: "meter", "year_built","square_feet", "floor_count" 
energy = energy.drop(columns=["meter", "year_built",
                              "square_feet", "floor_count"], axis=1)
del weather # удалим фрейм данных "weather"
# фрейм данных «energy» оптимизируем в соответствии с условиями функции «def reduce_mem_usage (df)»
energy = reduce_mem_usage(energy)
print (energy.info())#Посмотрим сводную информацию методом info()
# в таблице 432456 строки (0 to 432455)
# 12 столбцов - количество ненулевых значений (non-null)

Потребление памяти меньше на 28.46 Мб (минус 71.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432456 entries, 0 to 432455
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           432456 non-null  datetime64[ns]
 1   site_id             432456 non-null  int8          
 2   building_id         432456 non-null  int8          
 3   meter_reading       432456 non-null  float16       
 4   primary_use         432456 non-null  category      
 5   air_temperature     432312 non-null  float16       
 6   cloud_coverage      243994 non-null  float16       
 7   dew_temperature     432312 non-null  float16       
 8   precip_depth_1_hr   432408 non-null  float16       
 9   sea_level_pressure  428287 non-null  float16       
 10  wind_direction      420144 non-null  float16       
 11  wind_speed          432456 non-null  float16       
dtypes: category(1), datetime64[ns](1)

### Обогащение данных по датам и логарифмирование

In [14]:
# Вызовим функцию "add_dates_log" по параметру "df"
# преобразуем массив значений по "hour", "weekday", "week", "month" в целочисленный 8 бит
# преобразуем массив значений по "date" в формат DateTime
def add_dates_log (df):
    df["hour"] = df["timestamp"].dt.hour.astype("int8")
    df["weekday"] = df["timestamp"].dt.weekday.astype("int8")
    df["week"] = df["timestamp"].dt.isocalendar().week.astype("int8")
    df["month"] = df["timestamp"].dt.month.astype("int8")
    df["date"] = pd.to_datetime(df["timestamp"].dt.date)
# вычисляем соответсвие параметру "df" по "weekday", "week", "month" 
    for weekday in range(0,7):
        df['is_wday' + str(weekday)] = df['weekday'].isin([weekday]).astype("int8")
    for week in range(1,54):
        df['is_w' + str(week)] = df['week'].isin([week]).astype("int8")
    for month in range(1,13):
        df['is_m' + str(month)] = df['month'].isin([month]).astype("int8")
#с помощью функции date_range() создадим диапазон дат с 2015-12-31 до 2018-06-01 
    dates_range = pd.date_range(start='2015-12-31', end='2018-06-01')
#с помощью функции calendar().holidays выделим выходные дни      
    us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
    df['is_holiday'] = df['date'].isin(us_holidays).astype("int8")
# произведем логарифмирование данных по столбцу "meter_reading"   
    if "meter_reading" in df.columns:
        df["meter_reading_log"] = np.log(df["meter_reading"] + 1)
    return df # завершаем выполнение вызова функции "df"

In [15]:
# вычислим фрейм данных "energy" в соответствии с условиями функции "add_dates_log"
energy = add_dates_log(energy)
print(energy)

                 timestamp  site_id  building_id  meter_reading  \
0      2016-01-01 00:00:00        0            0       0.000000   
1      2016-01-01 00:00:00        0            1       0.000000   
2      2016-01-01 00:00:00        0            2       0.000000   
3      2016-01-01 00:00:00        0            3       0.000000   
4      2016-01-01 00:00:00        0            4       0.000000   
...                    ...      ...          ...            ...   
432451 2016-12-31 23:00:00        0           45       2.867188   
432452 2016-12-31 23:00:00        0           46       2.396484   
432453 2016-12-31 23:00:00        0           47     209.625000   
432454 2016-12-31 23:00:00        0           48      25.718750   
432455 2016-12-31 23:00:00        0           49     131.000000   

                primary_use  air_temperature  cloud_coverage  dew_temperature  \
0                 Education         25.00000             6.0             20.0   
1                 Education      

### Расчет моделей линейной регрессии

In [16]:
# сгенерируем списки "hours" (0-24) и "buildings" (0-50)
hours = range(0, 24)
buildings = range(0, energy["building_id"].max() + 1)
print(buildings)

range(0, 50)


In [17]:
# Вызовим функцию "calculate_model_coeffs" по параметру "columns"
def calculate_model_coeffs (columns):
    energy_train_lr = pd.DataFrame(energy, columns=columns)
    # заменим значения NaN на 0 во всех столбцах с помощью "fillna(0)"
    energy_train_lr = energy_train_lr.fillna(0)
    # создадим параметр "coeffs" по колличеству домов "buildings"
    coeffs = [[]]*len(buildings)
    # проработаем фрейм данных "buildings"
    for building in buildings:
        # создадим параметр "coeffs[building]" по колличеству домов "buildings" и времени "hours"
        coeffs[building] = [[]]*len(hours)
        # рассчитаем данные по каждому часу каждого дома
        energy_train_b = energy_train_lr[energy_train_lr["building_id"]==building]
        # рассчитаем энергопотребление по каждому часу        
        for hour in hours:
            energy_train_bh = energy_train_b[energy_train_b["hour"]==hour]
            y = energy_train_bh["meter_reading_log"]
            x = energy_train_bh.drop(labels=["meter_reading_log",
                "hour", "building_id"], axis=1)
            # создаем модель линейной регрессии
            model = LinearRegression(fit_intercept=False).fit(x, y)
            coeffs[building][hour] = model.coef_
            # объединим для массива данных "coeffs[building][hour]" и "model.intercept_"
            coeffs[building][hour] = np.append(coeffs[building][hour], model.intercept_)
    return coeffs  # завершаем выполнение вызова параметра "coeffs"

In [18]:
# создадим миссив данных "lr_columns_weather" (погода)
lr_columns_weather = ["meter_reading_log", "hour", "building_id",
             "air_temperature", "dew_temperature",
             "sea_level_pressure", "wind_speed", "cloud_coverage",
             "air_temperature_diff1", "air_temperature_diff2"]
# фрейм данных «lr_columns_weather» оптимизируем в соответствии с условиями функции "calculate_model_coeffs" (lr_columns_weather)
energy_lr_w = calculate_model_coeffs(lr_columns_weather)
print(energy_lr_w)

[[array([ 0.01224806,  0.26244469,  0.00090143, -0.22901475, -0.27209974,
        0.        ,  0.        ,  0.        ]), array([ 0.08061458,  0.17153483, -0.0005469 , -0.25012013,  0.09058016,
        0.        ,  0.        ,  0.        ]), array([-1.95729659e-02,  2.39169228e-01,  2.21950642e-04, -1.90606936e-01,
        1.38134816e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]), array([ 0.07356526,  0.1710531 , -0.00038357, -0.27999933,  0.0197059 ,
        0.        ,  0.        ,  0.        ]), array([ 0.0666137 ,  0.17433917, -0.00078836, -0.1290057 ,  0.16533339,
        0.        ,  0.        ,  0.        ]), array([ 0.2638186 ,  0.0074849 , -0.00193932, -0.07341998,  0.02041617,
        0.        ,  0.        ,  0.        ]), array([ 0.31254931, -0.01738681, -0.00198141, -0.04986942, -0.1100578 ,
        0.        ,  0.        ,  0.        ]), array([ 0.28038029, -0.0323915 , -0.0012204 , -0.16749705,  0.03980388,
        0.        ,  0.        ,  0.        ]), array(

In [19]:
# создадим миссив данных "lr_columns_days" (день)
lr_columns_days = ["meter_reading_log", "hour", "building_id",
                   "is_holiday"]
# рассчитаем данные для каждого дня
for wday in range(0,7):
    lr_columns_days.append("is_wday" + str(wday))
# фрейм данных «lr_columns_d» оптимизируем в соответствии с условиями функции "calculate_model_coeffs" (lr_columns_days)
energy_lr_d = calculate_model_coeffs(lr_columns_days)
print (energy_lr_d[0])

[array([0.46404106, 3.28496382, 3.36222957, 3.3816857 , 3.3443664 ,
       3.27391001, 3.39121462, 3.34728065, 0.        ]), array([0.48017462, 3.28346808, 3.35539363, 3.37800481, 3.35104232,
       3.27521747, 3.39460495, 3.3534405 , 0.        ]), array([0.46185674, 3.28668515, 3.35389123, 3.38176082, 3.35642763,
       3.27863571, 3.39770047, 3.34743089, 0.        ]), array([0.44014085, 3.29268837, 3.357497  , 3.37988281, 3.35429116,
       3.27938148, 3.40197524, 3.35554387, 0.        ]), array([0.42982065, 3.29820924, 3.3500601 , 3.38123498, 3.35306234,
       3.28102387, 3.3957842 , 3.35261418, 0.        ]), array([0.42599252, 3.29692168, 3.35351563, 3.37169471, 3.35614077,
       3.27969427, 3.393352  , 3.42608173, 0.        ]), array([0.44164408, 3.29796979, 3.34044471, 3.36816406, 3.35178329,
       3.27416555, 3.39401533, 3.35441707, 0.        ]), array([0.4679073 , 3.30232197, 3.36095252, 3.37973257, 3.36690323,
       3.27825998, 3.39224646, 3.35366587, 0.        ]), array([

In [20]:
# создадим миссив данных "lr_columns_weeks" (недели)
lr_columns_weeks = ["meter_reading_log", "hour", "building_id"]
# рассчитаем данные для каждой недели
for week in range(1,54):
    lr_columns_weeks.append("is_w" + str(week))
# фрейм данных «lr_columns_ww» оптимизируем в соответствии с условиями функции "calculate_model_coeffs" (lr_columns_weeks)
energy_lr_ww = calculate_model_coeffs(lr_columns_weeks)
print (energy_lr_ww[0])

[array([-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  1.57924107e+00,
        5.29296875e+00,  5.34988839e+00,  5.54185268e+00,  5.63169643e+00,
        5.51004464e+00,  5.50055804e+00,  5.49218750e+00,  5.68303571e+00,
        5.67578125e+00,  5.65066964e+00,  5.64118304e+00,  5.67354911e+00,
        5.74162946e+00,  5.54687500e+00,  5.50892857e+00,  5.52232143e+00,
        5.49330357e+00,  5.47712054e+00,  5.52120536e+00,  5.51450893e+00,
        5.50111607e+00,  5.50223214e+00,  5.36607143e+00,  5.34988839e+00,
        5.27120536e+00,  5.23046875e+00,  5.41294643e+00,  5.25781250e+00,
        4.87667411e+00,  5.05580357e+00,  5.31194196e+00,  5.42773438e+00,
       -2.05116020e-15, 

In [21]:
# создадим миссив данных "lr_columns_monthes" (месяца)
lr_columns_monthes = ["meter_reading_log", "hour", "building_id"]
# рассчитаем данные для каждой месяца
for month in range(1,13):
    lr_columns_monthes.append("is_m" + str(month))
# фрейм данных «lr_columns_m» оптимизируем в соответствии с условиями функции "calculate_model_coeffs" (lr_columns_monthes)    
energy_lr_m = calculate_model_coeffs(lr_columns_monthes)
print (energy_lr_m[0])

[array([0.        , 0.        , 0.        , 0.        , 1.8859627 ,
       5.52747396, 5.61504536, 5.63634073, 5.50364583, 5.47013609,
       5.31289062, 5.16796875, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 1.89024698,
       5.53242188, 5.61983367, 5.63961694, 5.50859375, 5.46610383,
       5.30898438, 5.16847278, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 1.890625  ,
       5.53958333, 5.6155494 , 5.64642137, 5.51419271, 5.46396169,
       5.30351563, 5.17401714, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 1.89264113,
       5.54205729, 5.60370464, 5.64629536, 5.51614583, 5.46950605,
       5.31236979, 5.19014617, 0.        ]), array([0.        , 0.        , 0.        , 0.        , 1.89352319,
       5.54583333, 5.59122984, 5.63293851, 5.52265625, 5.47101815,
       5.31549479, 5.18157762, 0.        ]), array([ 1.17817540e-01, -2.60264971e-18, -2.51729360e-18, -2.53372581e-18,
        1.89705141e+00,  5.55

### Удаление обучающих данных

In [22]:
del energy # удаляем обучающие данные "energy"

### Подготовка данных для расчетов

In [23]:
# загрузим данные для расчетов
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz")
weather = weather[weather["site_id"] == 0] # используем данные с условием "site_id" равен 0

In [35]:
results = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/test.csv.gz")

In [36]:
# используем данные по 50 зданиям 
results = results[(results["building_id"] < 50) & (results["meter"] == 0)]
# объединим данные по "results" и "buildings"
results = pd.merge(left=results, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
del buildings # удалим массив данных "buildings"
print (results.info())#Посмотрим сводную информацию методом info()
# в таблице 876000 строк (0 to 875999)
# 9 столбцов - количество ненулевых значений (non-null)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876000 entries, 0 to 875999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   row_id       876000 non-null  int64  
 1   building_id  876000 non-null  int64  
 2   meter        876000 non-null  int64  
 3   timestamp    876000 non-null  object 
 4   site_id      876000 non-null  int64  
 5   primary_use  876000 non-null  object 
 6   square_feet  876000 non-null  int64  
 7   year_built   876000 non-null  float64
 8   floor_count  0 non-null       float64
dtypes: float64(2), int64(5), object(2)
memory usage: 66.8+ MB
None


### Интерполяция значений и обогащение погодных данных: только для 1 города

In [37]:
# фрейм данных «weather» оптимизируем в соответствии с условиями функции "weather_interpolate_diff(weather)"
weather = weather_interpolate_diff(weather)

### Объединение данных по погоде

In [38]:
# проиндексируем в таблицах данных "results" и "weather" по столбцам: "timestamp", "site_id"
results = results.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
# объединим массивы данных "results" и "weather"
# с помощью параметра how="left" мы указали, что главным является датафрейм слева, название столбца – «results».
results = pd.merge(left=results, right=weather, how="left",
                  left_index=True, right_index=True)
# сбросим индекс по данным "results" 
results.reset_index(inplace=True)
# сгруппируем данные по столбцам: "meter", "site_id", "year_built","square_feet", "floor_count"
results = results.drop(columns=["meter", "site_id", "year_built",
                              "square_feet", "floor_count"], axis=1)
del weather # удалим массив данных "weather"
# фрейм данных «results» оптимизируем в соответствии с условиями функции «def reduce_mereduce_mem_usage(results)»
results = reduce_mem_usage(results)
print (results.info())#Посмотрим сводную информацию методом info()
# в таблице 876000 строк (0 to 875999)
# 13 столбцов - количество ненулевых значений (non-null)

Потребление памяти меньше на 60.15 Мб (минус 69.2 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              876000 non-null  datetime64[ns]
 1   row_id                 876000 non-null  int32         
 2   building_id            876000 non-null  int8          
 3   primary_use            876000 non-null  category      
 4   air_temperature        876000 non-null  float16       
 5   cloud_coverage         876000 non-null  float16       
 6   dew_temperature        876000 non-null  float16       
 7   precip_depth_1_hr      874500 non-null  float16       
 8   sea_level_pressure     876000 non-null  float16       
 9   wind_direction         852150 non-null  float16       
 10  wind_speed             876000 non-null  float16       
 11  air_temperature_diff1  876000 non-null  float16    

### Обогащение данных по дате

In [39]:
# фрейм данных «results» оптимизируем в соответствии с условиями функции «add_dates_log(results)»
results = add_dates_log(results)

### Расчет финальных показателей, только энергопотребление, только 50 первых зданий
Потребуется ~6 Гб оперативной памяти. Если память компьютера ограничена, имеет смысл провести расчеты по частям: например, по 10 зданий

In [40]:
# Вызовим функцию "calculate_model" по параметрам "x", "model", "columns"
def calculate_model (x, model, columns):
    return (np.sum([x[col] * model[i] for i,col in enumerate(columns[3:])])
            + model[len(columns)-3])

In [41]:
# Вызовим функцию "calculate_ensemble" по параметру "x"
def calculate_ensemble (x):
    lr = -1
    lr_w = calculate_model(x, 
            energy_lr_w[x.building_id][x.hour], lr_columns_weather)
    lr_d = calculate_model(x, 
            energy_lr_d[x.building_id][x.hour], lr_columns_days)
    lr_ww = calculate_model(x, 
            energy_lr_ww[x.building_id][x.hour], lr_columns_weeks)
    lr_m = calculate_model(x, 
            energy_lr_m[x.building_id][x.hour], lr_columns_monthes)
    lr = np.exp((lr_w*3 + lr_d*3 + lr_ww + lr_m)/8)
    if lr < 0 or lr != lr or lr*lr == lr:
        lr = 0
    x["meter_reading"] = lr
    return x
# преобразуем массив данных "results" по условиям функции "calculate_ensemble"
results = results.apply(calculate_ensemble, axis=1, result_type="expand")

### Усечение данных до требуемого формата: row_id, meter_reading - и выгрузка результата

In [42]:
results_ready = pd.DataFrame(results, columns=["row_id", "meter_reading"])
print (results_ready.info()) #Посмотрим сводную информацию методом info()
# в таблице 876000 строк (0 to 875999)
# 2 столбца - количество ненулевых значений (non-null)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   row_id         876000 non-null  int64  
 1   meter_reading  876000 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 13.4 MB
None


In [43]:
print (len(results_ready[results_ready["meter_reading"]>0]))

876000


### Загрузка всех данных для заполнения их нулями

In [45]:
results = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/test.csv.gz",
                     usecols=["row_id"])

IncompleteRead: IncompleteRead(80685575 bytes read, 82299514 more expected)

In [32]:
results = pd.merge(left=results, right=results_ready, how="left",
                  left_on="row_id", right_on="row_id")
results.fillna(value=0, inplace=True)
print (results.info()) #Посмотрим сводную информацию методом info()
# в таблице 41697600 строк (0 to 41697599)
# 2 столбца - количество ненулевых значений (non-null)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
 #   Column         Dtype  
---  ------         -----  
 0   row_id         int64  
 1   meter_reading  float64
dtypes: float64(1), int64(1)
memory usage: 954.4 MB
None


### Выгрузка результатов в CSV файл

In [33]:
results.to_csv("submission.csv",index=False)

### Освобождение памяти

In [34]:
del results
del results_ready